In [ ]:
#This code builds a temporal graph neural network (GCN + GRU + GAT) in PyTorch Geometric to perform binary classification on nodes from a dataset, training it for 5 epochs with BCE loss.
import pandas as pd
import torch
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal
from torch_geometric.nn import GCNConv, GATConv
import torch.nn as nn
import torch.nn.functional as F
df = pd.read_csv("retrain_validation10.csv")
node_mapping = {node: i for i, node in enumerate(
    pd.concat([df['author'], df['id']]).unique()
)}
sources = df['author'].map(node_mapping).values
targets = df['id'].map(node_mapping).values
edge_index = torch.tensor([sources, targets], dtype=torch.long)
feature_cols = [
    "toxicity_probability_self",
    "score_f",
    "hate_score_self",
    "hate_score_ctx"
]
features = torch.tensor(df[feature_cols].values, dtype=torch.float)
labels = torch.tensor(df['class_self'].values, dtype=torch.float)
dataset = DynamicGraphTemporalSignal(
    edge_indices=[edge_index],
    edge_weights=[None],
    features=[features],
    targets=[labels]
)
class GNNPoC(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.gcn = GCNConv(in_channels, hidden_channels)
        self.gru = nn.GRU(hidden_channels, hidden_channels, batch_first=True)
        self.gat = GATConv(hidden_channels, hidden_channels, heads=1)
        self.fc = nn.Linear(hidden_channels, out_channels)
     def forward(self, x, edge_index, h=None):
        # Spatial
        x = F.relu(self.gcn(x, edge_index))
        # Temporal
        x_seq = x.unsqueeze(0)
        out, h = self.gru(x_seq, h)
        x = out.squeeze(0)
        # Attention
        x = F.relu(self.gat(x, edge_index))
        return torch.sigmoid(self.fc(x)), h
model = GNNPoC(in_channels=features.size(1), hidden_channels=16, out_channels=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.BCELoss()  # binary classification
for epoch in range(5):
    for snapshot in dataset:
        x, edge_index, _, y = snapshot
        optimizer.zero_grad()
        y_pred, _ = model(x, edge_index)
        loss = loss_fn(y_pred.view(-1), y.float())
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch}, Loss {loss.item():.4f}")

In [ ]:
import pandas as pd
import torch
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal
from torch_geometric.nn import GCNConv, GATConv
import torch.nn as nn
import torch.nn.functional as F
import glob
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
# Loading multiple CVS as snapshots
csv_files = sorted(glob.glob("snapshots/*.csv"))
edge_indices, features_list, labels_list = [], [], []
node_mapping = {}
node_counter = 0
for file in csv_files:
df = pd.read_csv(file)
for node in pd.concat([df['author'], df['id']]).unique():
    if node not in node_mapping:
        node_mapping[node] = node_counter
        node_counter += 1
sources = df['author'].map(node_mapping).values
targets = df['id'].map(node_mapping).values
edge_index = torch.tensor([sources, targets], dtype=torch.long)
edge_indices.append(edge_index)
feature_cols = [
    "toxicity_probability_self",
    "score_f",
    "hate_score_self",
    "hate_score_ctx"
]
features = torch.tensor(df[feature_cols].values, dtype=torch.float)
features_list.append(features)

#Labels based on score_bin5: >3 = toxic (1), else = non-toxic (0)
labels = torch.tensor((df['score_bin5'].values > 3).astype(int), dtype=torch.float)
labels_list.append(labels)
dataset = DynamicGraphTemporalSignal(
edge_indices=edge_indices,
edge_weights=[None] * len(edge_indices),
features=features_list,
targets=labels_list
)
#GNN model
class GNNPoC(nn.Module):
def **init**(self, in_channels, hidden_channels, out_channels):
super().**init**()
self.gcn = GCNConv(in_channels, hidden_channels)
self.gru = nn.GRU(hidden_channels, hidden_channels, batch_first=True)
self.gat = GATConv(hidden_channels, hidden_channels, heads=1)
self.fc = nn.Linear(hidden_channels, out_channels)

def forward(self, x, edge_index, h=None):
    x = F.relu(self.gcn(x, edge_index))
    x_seq = x.unsqueeze(0)
    out, h = self.gru(x_seq, h)
    x = out.squeeze(0)
    x = F.relu(self.gat(x, edge_index))
    return torch.sigmoid(self.fc(x)), h

# Training + Evaluation loop

model = GNNPoC(in_channels=features_list[0].size(1), hidden_channels=16, out_channels=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.BCELoss()

for epoch in range(5):
h = None
total_loss = 0
all_preds, all_labels = [], []

for snapshot in dataset:
    x, edge_index, _, y = snapshot
    optimizer.zero_grad()
    y_pred, h = model(x, edge_index, h)

    loss = loss_fn(y_pred.view(-1), y.float())
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    # Store predictions for metrics
    all_preds.extend(y_pred.view(-1).detach().numpy())
    all_labels.extend(y.numpy())

# Convert predictions to binary
binary_preds = [1 if p > 0.5 else 0 for p in all_preds]

# Compute metrics
f1 = f1_score(all_labels, binary_preds, zero_division=0)
try:
    auc = roc_auc_score(all_labels, all_preds)
except ValueError:
    auc = float('nan')
cm = confusion_matrix(all_labels, binary_preds)

print(f"Epoch {epoch+1}")
print(f"  Avg Loss: {total_loss/len(dataset):.4f}")
print(f"  F1 Score: {f1:.4f}, ROC-AUC: {auc:.4f}")
print(f"  Confusion Matrix:\n{cm}")